<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2023-09-01 15:48:07


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv'
df_mypf = read_file(path_mypf)


def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

In [5]:
df_mypf = df_mypf[df_mypf['InPortfolio'].isnull()]
cols = ['Symbol', 'Criteria', 'Strategy', 'LatestQtr', 'Target','StarStock', 'BizCheck', 'FinCheck', 'TechCheck', 'Conviction', 'Category', 'Stock']
df_mypf = df_mypf[cols]
all_prospects = get_stock_ids(df_mypf)

In [6]:
def print_distribution():

  # star stocks
  print('Star Prospects:')
  print(df_mypf['StarStock'].value_counts())
  print('-------')

  # v40/v40n stocks
  print('V40/V40N Prospects:')
  print(df_mypf['Criteria'].value_counts())
  print('-------')

  # latest quarter
  print('Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())
  print('-------')

  # star + v40/v40n stocks
  print('Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())
  print('-------')

print_distribution()

Star Prospects:
1.0    66
0.0    64
Name: StarStock, dtype: int64
-------
V40/V40N Prospects:
V40N    29
V40     26
V200     2
SS       1
Name: Criteria, dtype: int64
-------
Latest Qtr:
1    124
0      7
Name: LatestQtr, dtype: int64
-------
Star + V40/V40N Stocks:
1.0    30
0.0    25
Name: StarStock, dtype: int64
-------


In [7]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_features(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK', 'ABBOTINDIA', 'JCHAC',
              'NESTLEIND', 'PGHH', 'EQUITASBNK', 'SBICARD', 'ASTRAZEN', 'ICICIGI', 'RELAXO']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:

    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
    current_roce = float(test_list[test_list.index('ROCE')+1:test_list.index('ROCE')+2][0])
    current_roe = float(test_list[test_list.index('ROE')+1:test_list.index('ROE')+2][0])
  except ValueError as ve:
    current_pe = 1000
    current_roce = 1000
    current_roe = 1000

  return current_pe, current_roce, current_roe

In [9]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),0)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      #print(stock_id)
      current_pe, current_roce, current_roe = get_current_features(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Curr_PE'] = current_pe
      tmp['Curr_ROCE%'] = current_roce
      tmp['Curr_ROE%'] = current_roe
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()

In [10]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,RSI_14
Date,,,,,,
2023-08-03 00:00:00+05:30,19382.0,18217.0,6.39,^NSEI,19979.0,47.0
2023-08-10 00:00:00+05:30,19543.0,18271.0,6.96,^NSEI,19979.0,52.0
2023-08-18 00:00:00+05:30,19310.0,18314.0,5.44,^NSEI,19979.0,43.0
2023-08-25 00:00:00+05:30,19266.0,18346.0,5.01,^NSEI,19979.0,41.0
2023-09-01 00:00:00+05:30,19435.0,18374.0,5.78,^NSEI,19979.0,51.0


In [11]:
stock_ids = df_common_features['Stock'].values
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

In [12]:
df_prospects = pd.merge(df_common_features, df_mypf, on='Stock')

In [13]:
#df_common_features['Discount%PE'] = round((df_common_features['MedianPE']-df_common_features['Curr_PE'])*100/(df_common_features['MedianPE']),0)
df_prospects['Target'] = np.where(df_prospects['Strategy']== 'BTT', df_prospects["Target"], df_prospects['Max'])
df_prospects['Upside%Target'] = round((df_prospects['Target']-df_prospects['Close'])*100/(df_prospects['Close']),0)
df_prospects.drop([ '200_SMA', 'Symbol', 'Target', 'Max', 'Close'], axis=1, inplace=True)
df_prospects = df_prospects.set_index('Stock')

In [14]:
# top 10 prospects to invest from BTT stocks
df_prospects[df_prospects['Strategy'] == 'BTT'].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
ASIANPAINT.NS,6.77,48.0,67.1,35.2,27.7,V40,BTT,1,1.0,1.0,1,1.0,H,PAINTS,30.0
BAJFINANCE.NS,11.71,57.0,35.9,11.8,23.5,V40,BTT,1,1.0,1.0,1,1.0,H,FINANCE,25.0
ABBOTINDIA.NS,5.02,36.0,46.7,41.1,31.6,V40,BTT,1,1.0,1.0,1,NaN,M,PHARMA,21.0
AKZOINDIA.NS,15.91,48.0,33.9,34.7,26.1,V40,BTT,1,1.0,1.0,1,NaN,M,NaN,21.0
UJJIVANSFB.NS,47.88,57.0,1000.0,1000.0,1000.0,V40N,BTT,1,1.0,1.0,1,1.0,M,BANKS,12.0
COLPAL.NS,19.79,50.0,122.0,180.0,158.0,V40,BTT,1,1.0,0.0,1,1.0,H,FMCG,8.0


In [15]:
# prospects to invest from star + V40/V40N/VR + below 200 DMA
df_prospects[(df_prospects['Criteria'].notnull()) & (df_prospects['StarStock'] == 1) & (df_prospects['Dev%_200'] < 5)].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
VINATIORGA.NS,-2.07,61.0,43.4,30.4,22.60,V40N,NaN,1,1.0,1.0,NaN,NaN,M,NaN,26.0
RELIANCE.NS,-2.40,34.0,25.3,10.0,8.94,V40,ATH,1,1.0,1.0,1,1.0,H,REFINERIES,17.0
DABUR.NS,0.43,42.0,57.1,23.3,19.50,V40,NaN,1,1.0,0.0,1,1.0,H,FMCG,16.0
HINDUNILVR.NS,-2.50,36.0,56.8,26.6,20.50,V40,NaN,1,1.0,0.0,NaN,1.0,H,FMCG,10.0


In [16]:
# top 10 prospects to invest from star stocks
df_prospects[df_prospects['StarStock'] == 1].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
ADANIGREEN.NS,-20.11,43.0,123.00,7.81,22.80,NaN,NaN,1,1.0,0.0,1,0.0,L,NaN,213.0
TATAELXSI.NS,7.37,53.0,444.00,60.00,39.30,V40N,NaN,1,1.0,1.0,NaN,NaN,M,NaN,46.0
SBICARD.NS,4.00,48.0,35.80,13.00,25.70,NaN,NaN,1,1.0,0.0,NaN,1.0,M,NaN,35.0
MUTHOOTFIN.NS,14.87,43.0,13.40,12.10,17.90,NaN,NaN,1,1.0,0.0,NaN,0.0,L,FINANCE,31.0
ASIANPAINT.NS,6.77,48.0,67.10,35.20,27.70,V40,BTT,1,1.0,1.0,1,1.0,H,PAINTS,30.0
KANSAINER.NS,18.88,65.0,50.90,14.50,10.70,V40N,NaN,1,1.0,1.0,NaN,NaN,M,NaN,28.0
BPCL.NS,-0.52,37.0,3.67,6.86,6.34,NaN,NaN,1,1.0,0.0,1,NaN,L,NaN,27.0
VINATIORGA.NS,-2.07,61.0,43.40,30.40,22.60,V40N,NaN,1,1.0,1.0,NaN,NaN,M,NaN,26.0
BAJFINANCE.NS,11.71,57.0,35.90,11.80,23.50,V40,BTT,1,1.0,1.0,1,1.0,H,FINANCE,25.0


In [17]:
# top 10 prospects to invest from  V40/V40N/VR
df_prospects[df_prospects['Criteria'].notnull()].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
JCHAC.NS,17.99,69.0,1000.0,1000.00,1000.00,V40N,NaN,1,0.0,1.0,NaN,NaN,M,NaN,146.0
LALPATHLAB.NS,3.73,43.0,69.5,18.50,15.00,V40N,NaN,1,0.0,1.0,NaN,NaN,M,NaN,89.0
SFL.NS,-2.97,49.0,52.4,13.60,12.50,V40N,NaN,1,0.0,1.0,NaN,NaN,M,NaN,75.0
SYMPHONY.NS,-6.29,52.0,54.9,14.80,13.50,V40N,NaN,1,0.0,1.0,NaN,NaN,M,NaN,66.0
WHIRLPOOL.NS,14.82,76.0,98.5,8.78,6.37,V40,NaN,1,0.0,0.0,NaN,1.0,M,NaN,63.0
AVANTIFEED.NS,15.30,70.0,18.4,20.70,13.40,V200,NaN,1,0.0,1.0,1,NaN,M,NaN,62.0
PFIZER.NS,-3.35,40.0,114.0,27.20,19.80,V40,NaN,1,0.0,0.0,NaN,0.0,L,PHARMA,57.0
TATAELXSI.NS,7.37,53.0,444.0,60.00,39.30,V40N,NaN,1,1.0,1.0,NaN,NaN,M,NaN,46.0
TTKPRESTIG.NS,5.53,62.0,44.9,18.20,13.80,V40N,NaN,1,0.0,1.0,NaN,NaN,M,NaN,44.0


In [18]:
# top 10 prospects to invest below 200 DMA
df_prospects[df_prospects['Dev%_200'] < 0].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
ADANIGREEN.NS,-20.11,43.0,123.00,7.81,22.80,NaN,NaN,1,1.0,0.0,1,0.0,L,NaN,213.0
ALKYLAMINE.NS,-1.93,62.0,66.90,41.30,41.40,NaN,NaN,0,NaN,NaN,1,NaN,M,CHEMICALS,84.0
SFL.NS,-2.97,49.0,52.40,13.60,12.50,V40N,NaN,1,0.0,1.0,NaN,NaN,M,NaN,75.0
ADANIENT.NS,-5.07,46.0,100.00,10.10,9.63,NaN,NaN,1,0.0,0.0,1,1.0,M,NaN,70.0
SYMPHONY.NS,-6.29,52.0,54.90,14.80,13.50,V40N,NaN,1,0.0,1.0,NaN,NaN,M,NaN,66.0
PFIZER.NS,-3.35,40.0,114.00,27.20,19.80,V40,NaN,1,0.0,0.0,NaN,0.0,L,PHARMA,57.0
UPL.NS,-12.61,51.0,16.60,15.70,13.40,NaN,NaN,1,0.0,0.0,NaN,NaN,L,NaN,36.0
SHREECEM.NS,-1.11,53.0,55.70,8.53,6.98,NaN,NaN,1,0.0,0.0,NaN,NaN,L,NaN,30.0
BPCL.NS,-0.52,37.0,3.67,6.86,6.34,NaN,NaN,1,1.0,0.0,1,NaN,L,NaN,27.0


In [19]:
# star + v40/v40n prospects (for breakout)

tmp_df = df_prospects[df_prospects['StarStock'] == 1]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=True)

qualified stocks: 10


,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
ANGELONE.NS,39.60,73.0,17.3,44.00,47.1,V40N,NaN,1,1.0,1.0,1,NaN,M,FINANCE,0.0
AXISBANK.NS,8.80,62.0,13.4,6.16,14.5,V40,NaN,1,1.0,0.0,1,1.0,M,BANKS,0.0
CAPLIPOINT.NS,45.40,68.0,21.2,26.80,22.4,V40N,NaN,1,1.0,1.0,1,NaN,M,NaN,0.0
CERA.NS,44.36,80.0,53.6,25.80,19.4,V40N,NaN,1,1.0,1.0,1,1.0,H,CERAMICS,0.0
POLYCAB.NS,56.78,72.0,53.8,27.10,20.0,V40N,NaN,1,1.0,1.0,NaN,1.0,M,CABLES,0.0
ICICIBANK.NS,7.75,56.0,18.3,6.35,17.2,V40,NaN,1,1.0,0.0,NaN,1.0,H,BANKS,1.0
TITAN.NS,16.43,67.0,86.0,25.10,30.8,V40,NaN,1,1.0,0.0,NaN,1.0,H,JEWELLERY,1.0
3MINDIA.NS,27.27,66.0,70.9,31.40,23.4,V40N,NaN,1,1.0,1.0,NaN,NaN,M,NaN,2.0
MARICO.NS,10.34,57.0,54.3,43.00,35.8,V40,NaN,1,1.0,0.0,NaN,1.0,M,FMCG,3.0


In [20]:
# prospects for interactive analysis

df_prospects

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
3MINDIA.NS,27.27,66.0,70.9,31.40,23.40,V40N,NaN,1,1.0,1.0,NaN,NaN,M,NaN,2.0
ABBOTINDIA.NS,5.02,36.0,46.7,41.10,31.60,V40,BTT,1,1.0,1.0,1,NaN,M,PHARMA,21.0
ACC.NS,1.53,62.0,44.5,10.00,7.06,NaN,NaN,0,0.0,0.0,1,0.0,L,CEMENT,35.0
ADANIENT.NS,-5.07,46.0,100.0,10.10,9.63,NaN,NaN,1,0.0,0.0,1,1.0,M,NaN,70.0
ADANIGREEN.NS,-20.11,43.0,123.0,7.81,22.80,NaN,NaN,1,1.0,0.0,1,0.0,L,NaN,213.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VIPIND.NS,2.54,51.0,56.4,27.80,29.00,V40N,NaN,1,0.0,1.0,NaN,NaN,M,NaN,13.0
VOLTAS.NS,7.02,71.0,87.0,9.74,4.42,NaN,NaN,1,0.0,0.0,NaN,0.0,L,NaN,51.0
WHIRLPOOL.NS,14.82,76.0,98.5,8.78,6.37,V40,NaN,1,0.0,0.0,NaN,1.0,M,NaN,63.0


In [21]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [22]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BERGEPAINT.NS')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,RSI_14
Date,,,,,,
2023-08-03 00:00:00+05:30,695.0,608.0,14.36,BERGEPAINT.NS,854.0,61.0
2023-08-10 00:00:00+05:30,693.0,610.0,13.63,BERGEPAINT.NS,854.0,55.0
2023-08-18 00:00:00+05:30,705.0,613.0,15.05,BERGEPAINT.NS,854.0,61.0
2023-08-25 00:00:00+05:30,704.0,616.0,14.34,BERGEPAINT.NS,854.0,57.0
2023-09-01 00:00:00+05:30,720.0,619.0,16.29,BERGEPAINT.NS,854.0,64.0
